## GENESIS API: Changes as of June 30, 2025

To improve the protection of your user data, the previously offered option of using the API via `GET` requests will be permanently deactivated on June 30, 2025. After this date, the GENESIS API will only be accessible using the `POST` method. The access data is sent in the header of the request and therefore cannot get into log files or be tapped during transport.
To help you with the changeover, we show executable code examples that use the POST method, starting with the **Python** programming language and the **Pandas** statistics package.

In [54]:
import requests
import pandas as pd
import io
import zipfile

We recommend using the **API token** to identify yourself to the web service. You will find this string in the [web service interface (API) menu](https://www-genesis.destatis.de/datenbank/online#modal=web-service-api) after logging in. It can be reset separately of your login data and therefore offers more control for shared projects, in training environments, etc. The token also does not contain any characters that may need to be masked and is therefore less prone to errors in use.
However, no writing access is possible after identification with a token. This includes changing the password via API or a table retrieval that is written to a queue or batch due to its size `(job=true)`. Such writing accesses must be identified by user name/email and password. The program examples described here can be used for both types of identification.
If you do not yet have access data for the GENESIS database, you can [register](https://www-genesis.destatis.de/datenbank/online#modal=register) easily and free of charge.

In [66]:
BASE_URL = 'https://www-genesis.destatis.de/genesisWS/rest/2020/'

TOKEN = "58b3_IHREN_TOKEN_EINFÜGEN_e6d032"

Regardless of the type of access data used, these must be transmitted in the request header of a POST request with the `‘Content-Type’: ‘application/x-www-form-urlencoded’`.

In [5]:
headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'username': TOKEN,
    'password': ""
    }

langPref = "de"

### Download a single table in Excel format
As an example, Individual consumption by purpose, 5-digit codes (`CC13A5`) of the monthly consumer price index are to be retrieved, namely all those belonging to `CC13-071 Purchase of vehicles`.
A documentation of the entire functional scope of the API can be found in the [textual interface description](https://www-genesis.destatis.de/datenbank/online/docs/GENESIS-Webservices_Introduction.pdf) (PDF, 899kB)

In [24]:
responseTable = requests.post(BASE_URL + 'data/tablefile',
    headers = headers,
    data = {
    'name': '61111-0004',
    'startyear': 2025,
    'transpose': "true",
    'classifyingvariable1': "CC13A5",
    'classifyingkey1': "CC13-071*",
    'compress': 'true',
    'format': 'xlsx',
    'language': langPref
    })

### Saving the file if the retrieval was successful

In [27]:
if responseTable.status_code == 200:
    with open("table.xlsx", "wb") as f:
        f.write(responseTable.content)
else:
    print("Antwort-Code: "+(responseTable.status_code))

## Handling large tables
Tables with more than 40,000 values (as of 24 March 2025) cannot be downloaded directly in dialog via API. Either the request must be sent to a queue (`job=true`) and retrieved later with `resultfile` or the request must be split.  
The table breaking down the Individual consumption by purpose, 10-digit codes (`CC13Z1`) based on 2020 already exceeds this limit at the beginning of 2025:

In [47]:
response = requests.post(BASE_URL + 'data/tablefile',
    headers = headers,
    data = {
    'name': '61111-0006',
    'startyear': 2020,
    "classifyingvariable1": "CC13Z1",
    'compress': 'true',
    'format': 'ffcsv',
    'language': langPref
})
# check whether the answer is text, binary or empty
print(response.text[0:120])

{"Ident":{"Service":"data","Method":"tablefile"},"Status":{"Code":98,"Content":"Die Tabelle ist zu gross, um im Dialog a


#### Split into two table retrievals along the time axis
The flat file csv format (`ffcsv`) is the method of choice for further processing in **Pandas**. These files can simply be merged one after the other. In many cases, the number of lines corresponds to the number of values. In the following example, the values for January 2020 to December 2023 are retrieved first, followed by the current margin (January 2024 and later).

In [ ]:
response1 = requests.post(BASE_URL + 'data/tablefile',
    headers = headers,
    data = {
    'name': '61111-0006',
    'startyear': 2020,
    'endyear': 2023,
    "classifyingvariable1": "CC13Z1",
    'compress': 'true',
    'format': 'ffcsv',
    'language': langPref
})

In [ ]:
response2 = requests.post(BASE_URL + 'data/tablefile',
    headers = headers,
    data = {
    'name': '61111-0006',
    'startyear': 2024,
    "classifyingvariable1": "CC13Z1",
    'compress': 'true',
    'format': 'ffcsv',
    'language': langPref
})

#### Read zipped file response directly without saving
GENESIS-Online delivers zipped csv and ffcsv files, whereby each ZIP archive contains exactly one csv file.  
The direct processing from the working memory is shown here as an example:

In [57]:
def csvUnZip(response):
    filebytes = io.BytesIO(response.content)
    zipFile = zipfile.ZipFile(filebytes)
    csvFile = zipFile.open(zipFile.namelist()[0])
    return csvFile

#### Parameters for importing the German-language version of the ffcsv files

In [58]:
def readCsv(csvFile):
    df = pd.read_csv(csvFile, delimiter=';', decimal=",", na_values=["...",".","-","/","x"])
    return df

#### Merge the two tablefile responses

In [70]:
df_combined = pd.concat([readCsv(csvUnZip(response1)),readCsv(csvUnZip(response2))])\
                .sort_values(by=["time","1_variable_attribute_code","3_variable_attribute_code"])

df_combined[["statistics_label","time","1_variable_attribute_label","3_variable_attribute_label","value"]]

,statistics_label,time,1_variable_attribute_label,3_variable_attribute_label,value
21299,Verbraucherpreisindex für Deutschland,2020,Januar,Reis,99.6
29259,Verbraucherpreisindex für Deutschland,2020,Januar,Reiszubereitung,100.0
5864,Verbraucherpreisindex für Deutschland,2020,Januar,Weizenmehl,100.9
15401,Verbraucherpreisindex für Deutschland,2020,Januar,"Grieß, Roggenmehl oder Ähnliches",101.3
1352,Verbraucherpreisindex für Deutschland,2020,Januar,Weißbrot,101.0
...,...,...,...,...,...
5042,Verbraucherpreisindex für Deutschland,2025,Februar,Passgebühr oder Ähnliches,120.7
4515,Verbraucherpreisindex für Deutschland,2025,Februar,Rechtsanwaltsgebühr oder Notargebühr,117.4
3584,Verbraucherpreisindex für Deutschland,2025,Februar,Bestattungsleistungen und Friedhofsgebühr,120.8
1652,Verbraucherpreisindex für Deutschland,2025,Februar,Kleinanzeige in einer Zeitung,119.2


### Create and retrieve batch job
For very large tables, retrieval via a queue (batch) is required. To do this, call `data/tablefile` with the parameter `job=true`. 
After a few minutes, the processing status of the requested results table can be found via `catalogue/results` and finally downloaded with `data/resultfile`.